In [1]:
import torch
import numpy as np

import os
os.chdir('..')
from torch_geometric.loader import DataLoader
from gnn_library.util import train, save, load, gen_train_input
from evaluate import evaluate_model, pp_output
import instance_generator as ig
import torch_converter as tc
import evaluate as ev
import osmnx as ox
from util import Dataset
from gnn_library.OBM_threshold_greedy import OBM_Threshold_Greedy
from experiment_parameters import *

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print("PyTorch has version {}".format(torch.__version__))
print('Using device:', device)

In [ ]:
# Hyperparam optimized for 10,6

args = {
    'processor':         'GENConv',
    'head':              'regression',     
    'num_layers':        3,
    'num_mlp_layers':    3,
    'aggr':              'max',
    'batch_size':        8,
    'node_feature_dim':  5,
    'edge_feature_dim':  1,
    'graph_feature_dim': 2,
    'hidden_dim':        32,
    'output_dim':        1,
    'dropout':           0.0306,
    'epochs':            64,
    'opt':               'adagrad',
    'opt_scheduler':     'none',
    'opt_restart':       0,
    'weight_decay':      5e-3,
    'lr':                0.0121,
    'device':            device,
    'noise':             0 # Set to 0 to not train on noisy features
}

train_loader, val_loader = gen_train_input(BASE_MODEL_TRAIN_CONFIG, args, seed=0)
_, _, _, GNN, _ = train(train_loader, val_loader, args)

In [ ]:
save(GNN, args, 'GNN2')

In [ ]:
GNN, args = load('GNN2', device)

## Box plot baseline evaluation

In [ ]:
import gnn_library.util as util
num_trials = 40
threshold = 0.35
batch_size = 500

models = [("GNN", GNN)]
t_greedy = OBM_Threshold_Greedy(threshold) 

rng = np.random.default_rng()

node_configs = EXPERIMENT_BASE_TESTING_REGIMES
graph_configs = ALL_TESTING_GRAPH_CONFIGS

data = {node_config: {} for node_config in node_configs}

for node_config in node_configs:
    for graph_config in graph_configs:
            print(graph_config)
            instances = ig.sample_instances(*node_config, num_trials, rng, args, **graph_config)

            for model_name, model in models:
                cr_ratios, _ = evaluate_model(
                    meta_model=None,
                    meta_model_type=None,
                    base_models=[model],
                    instances=instances,
                    batch_size=batch_size,
                    rng=rng,
                    num_realizations=5,
                    baselines=['greedy', 'lp_rounding'],
                )

                data[node_config][graph_config['graph_type']] = cr_ratios

                t_greedy_ratio, _ = evaluate_model(
                    meta_model=None,
                    meta_model_type=None,
                    base_models=[t_greedy],
                    instances=instances,
                    batch_size=batch_size,
                    rng=rng,
                    num_realizations=5,
                )

                # Threshold greedy is considered as a model here
                data[node_config][graph_config['graph_type']]['threshold_greedy'] = t_greedy_ratio['learned']


In [ ]:
filtered_data = {}
for graph_type, val in data[EXPERIMENT_BASE_TESTING_REGIMES[0]].items():
	if dict(graph_type) in util.graph_configs_main:
		filtered_data[graph_type] = val

In [ ]:
from util import _box_plots
_box_plots(filtered_data, lambda graph_type: f"GNN2_classify_{graph_type} {node_config[1]}x{node_config[0]}")